In [6]:
from google.colab import drive
drive.mount('/content/drive/')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

import pandas as pd
import numpy as np
import re
import os
import time
import json
import pickle
import pathlib
from glob import glob

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
os.chdir('/content/drive/MyDrive/Colab Notebooks')
import ongaku

In [8]:
BATCH_SIZE = 64
embedding_dim = 256 # 単語埋め込みの次元数
units = 512
vocab_size = 55 #テキストデータのサイズ
features_shape = 1000 #VGG16の特徴量の次元数（）
attention_features_shape = 49 # attentionの特徴量の次元数
max_length = 14

# 画像の特徴量抽出モデル
image_model = VGG16(include_top=False, weights='imagenet')
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(inputs=image_model.input, outputs=hidden_layer)
encoder = ongaku.Encoder(embedding_dim)
decoder = ongaku.Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
checkpoint_path = "./checkpoints/train"
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_path, max_to_keep=5)

# checkpoint_path の中の最後のチェックポイントを復元
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

In [9]:
pip install pydub

In [10]:
import numpy as np
import IPython
from pydub import AudioSegment
import wave

# 正弦波
def sine(freq, t):
  # freq: 周波数（数値）
  # t: 時刻（ベクトル）
  return np.sin(2.0*np.pi*freq*t)

# 三角波
def triangle(freq, t):
    return np.abs((2*t*freq-1/2)%2 - 1) * 2 -1

# 矩形波
def square(freq, t):
    # [0-pi] -> 1, [pi-2pi] -> -1
    return (np.ceil(t * freq * 2) % 2) * 2 - 1

# 25%パルス波
def pulse_quarter(freq, t):
    return (np.ceil(t * freq * 4 - 1) % 4 == 0) * 2 - 1

# 12.5%パルス波
def pulse_eighth(freq, t):
    return (np.ceil(t * freq * 8 - 1) % 8 == 0) * 2 - 1

# 音楽再生
def play_music(music):
  rate = 44100
  return IPython.display.Audio(music, rate=rate, autoplay=True)

# 十二平均率で周波数を求める
def note_to_freq(notestr):
    if notestr == 'R':
       return 0
    key, octave = notestr[:-1], notestr[-1] #スライスで文字列と数字を分ける
    keys = ['ド', 'ド#', 'レ', 'レ#', 'ミ', 'ファ', 'ファ#', 'ソ', 'ソ#', 'ラ', 'ラ#', 'シ']
    return 27.5 * 2 ** (int(octave) + (keys.index(key)-9)/12)

# メロディ作成
def make_mml(mml, sw):
  rate = 44100
  duration = 1.0
  music = np.array([])
  for s in mml:
    t = np.linspace(0, duration, int(rate * duration))
    f = note_to_freq(s)
    if sw == 1:
        x = sine(f, t)
    elif sw == 2:
        x = triangle(f, t)
    elif sw == 3:
        x = square(f, t)
    elif sw == 4:
        x = pulse_quarter(f, t)
    else:
        x = pulse_eighth(f, t)
    music = np.append(music, x)
  return music

# メロディ入力
def mml_input(sw, i, mml):
    sw = sw
    if i == 0:
      d = {'4': None, 'C': 'ド','D': 'レ','E': 'ミ','F': 'ファ','G': 'ソ','A': 'ラ','B': 'シ', 'm': None,'a': None,'u': None,'g': None,'s': None, '6': None, '7': None}
      tbl = str.maketrans(d)  # 辞書dをから変換テーブルを作成
      mml = [i.translate(tbl) for i in mml]
      mml = [i+'4' for i in mml if i!='R']
    elif i == 1:
      mml = [i.replace('Cm7', 'レ#4').replace('Cm', 'レ#4').replace('C#m7', 'ミ4').replace('C#m', 'ミ4').replace('C#', 'ファ4').replace('C', 'ミ4') for i in mml]
      mml = [i.replace('D#aug', 'ソ4').replace('D7', 'ファ#4').replace('Dm7', 'ファ4').replace('D#m7', 'ファ#4').replace('D#m', 'ファ#4').replace('D#', 'ソ4').replace('D', 'ファ#4') for i in mml]
      mml = [i.replace('E7', 'ソ#4').replace('Em', 'ソ4').replace('E', 'ソ#4') for i in mml]
      mml = [i.replace('F7', 'ラ4').replace('Fm7', 'ラ4').replace('Fm', 'ソ#4').replace('F#m7', 'ラ4').replace('F#m', 'ラ4').replace('F#', 'ラ#4').replace('F', 'ラ4') for i in mml]
      mml = [i.replace('Gm7', 'ラ#4').replace('G#m', 'シ4').replace('G#7', 'ド5').replace('G#', 'ド5').replace('G', 'シ4') for i in mml]
      mml = [i.replace('A7', 'ド#5').replace('Am6', 'ド5').replace('Am', 'ド5').replace('Asus4', 'レ5').replace('A#m7', 'ド#5').replace('A#m', 'ド#5').replace('A#', 'レ5').replace('A', 'ド#5') for i in mml]
      mml = [i.replace('B7', 'レ#5').replace('Bm', 'レ5').replace('Baug', 'レ#5').replace('B', 'レ#5')  for i in mml]
    elif i == 2:
      mml = [i.replace('Cm7', 'ソ4').replace('Cm', 'ソ4').replace('C#m7', 'ソ#4').replace('C#m', 'ソ#4').replace('C#', 'ソ#4').replace('C', 'ソ4') for i in mml]
      mml = [i.replace('D7', 'ラ4').replace('Dm7', 'ラ4').replace('D#m7', 'ラ#4').replace('D#m', 'ラ#4').replace('D#aug', 'シ4').replace('D#', 'ラ#4').replace('D', 'ラ4') for i in mml]
      mml = [i.replace('E7', 'シ4').replace('Em', 'シ4').replace('E', 'シ4') for i in mml]
      mml = [i.replace('F7', 'ド5').replace('Fm7', 'ド5').replace('F#m7', 'ド#5').replace('F#m', 'ド#5').replace('Fm', 'ド5').replace('F#', 'ド#5').replace('F', 'ド5') for i in mml]
      mml = [i.replace('Gm7', 'レ5').replace('G#m', 'レ#5').replace('G#7', 'レ#5').replace('G#', 'レ#5').replace('G', 'レ5') for i in mml]
      mml = [i.replace('A7', 'ミ5').replace('Am6', 'ミ5').replace('Am', 'ミ5').replace('Asus4', 'ミ5').replace('A#m7', 'ファ5').replace('A#m', 'ファ5').replace('A#', 'ファ5').replace('A', 'ミ5') for i in mml]
      mml = [i.replace('B7', 'ファ#5').replace('Bm', 'ファ#5').replace('Baug', 'ソ5').replace('B', 'ファ#5')  for i in mml]
    elif i ==3:
      mml = [i.replace('Cm7', 'ラ#4').replace('Cm', 'R').replace('C#m7', 'シ4').replace('C#m', 'R').replace('C#', 'R').replace('C', 'R') for i in mml]
      mml = [i.replace('D7', 'ド5').replace('Dm7', 'ド5').replace('D#m7', 'ド#5').replace('D#m', 'R').replace('D#aug', 'ド#5').replace('D#', 'R').replace('D', 'R') for i in mml]
      mml = [i.replace('E7', 'レ5').replace('Em', 'R').replace('E', 'R') for i in mml]
      mml = [i.replace('F7', 'レ#5').replace('Fm7', 'ミ5').replace('F#m7', 'ミ5').replace('F#m', 'R').replace('Fm', 'R').replace('F#', 'R').replace('F', 'R') for i in mml]
      mml = [i.replace('Gm7', 'ファ5').replace('G#m', 'R').replace('G#7', 'ファ#5').replace('G#', 'R').replace('G', 'R') for i in mml]
      mml = [i.replace('A7', 'ソ5').replace('Am6', 'ファ5').replace('Am', 'R').replace('Asus4', 'R').replace('A#m7', 'ソ#5').replace('A#m', 'R').replace('A#', 'R').replace('A', 'R') for i in mml]
      mml = [i.replace('B7', 'ラ5').replace('Bm', 'R').replace('Baug', 'R').replace('B', 'R')  for i in mml]
    m = make_mml(mml, sw)
    return m

# 楽曲保存
def do_save(music, fname):
    sampling_rate = 44100
    # 16ビットで量子化
    music = (music*32767).astype(np.int16)
    with wave.Wave_write(fname) as fp:
        fp.setframerate(sampling_rate)
        fp.setnchannels(1) # モノラル
        fp.setsampwidth(2) # 16ビット（バイト数を入力する）
        fp.writeframes(music.tobytes()) # バイナリ化
    print('保存しました')

# 保存するか尋ねる
def save_input(music):
    save = input('保存しますか？Y/N:')
    if save == 'Y':
        fname = input('名前を付けてください：')
        fname = fname + '.wav'
        do_save(music, fname)
        to_mp3(fname)
    else:
        exit()

def to_mp3(fname):
    # 読み込み
    sound = AudioSegment.from_wav(fname)
    rname = fname.rstrip('.wav') + '.mp3'
    # 変換
    sound.export(rname, format='mp3')
    return print('mp3に変換しました。')

def waon(mml_f):
  num = 4
  if '<end>' in mml_f:
    mml_f.remove('<end>')
  mml_f = [i.capitalize() for i in mml_f]
  mml_f = [i.replace('B♭ond', 'A#').replace('Bond#', 'B').replace('Donf#', 'D').replace('Gond', 'G').replace('D♭', 'C#').replace('E♭', 'D#').replace('G♭', 'F#').replace('A♭', 'G#').replace('B♭', 'A#') for i in mml_f]
  sw = int(input('音色を選んでください(やわらかい音:1～5:歪んだ音)：'))
  for i in range(num):
    mml = mml_f
    if i == 0:
      music = mml_input(sw, i, mml)
    else:
      m = mml_input(sw, i, mml)
      music = music + m
  return music * 0.1 #修正①和音でも音声ファイルに保存できるように修正

In [11]:
mml_f = ongaku.make_c()
music = waon(mml_f)
save_input(music)

画像のパス：/content/drive/MyDrive/demo/demo1.jpg
fm7|c|g|am|fm7|c|g|am|fm7|c|g|am|fm7|c
音色を選んでください(やわらかい音:1～5:歪んだ音)：3
保存しますか？Y/N:Y
名前を付けてください：test
保存しました
mp3に変換しました。
